In [ ]:
import ROOT

In [ ]:
import fastjet as fj
import fjext
import fjcontrib

import pythia8
import pythiafjext
import pythiaext

In [ ]:
# Generator.
pythia = pythia8.Pythia()
extra_s = ["Next:numberCount = 0", "Next:numberShowEvent = 0", "Next:numberShowInfo = 0", "Next:numberShowProcess = 0", "Stat:showProcessLevel = on"]
for s in extra_s:
    pythia.readString(s)
# Allow no substructure in e+- beams: normal for corrected LEP data.
pythia.readString("PDF:lepton = off")
# Process selection.
pythia.readString("WeakSingleBoson:ffbar2gmZ = on")
# Switch off all Z0 decays and then switch back on those to quarks.
pythia.readString("23:onMode = off");
pythia.readString("23:onIfAny = 1 2 3 4 5")
# LEP1 initialization at Z0 mass.
pythia.readString("Beams:idA =  11")
pythia.readString("Beams:idB = -11")
mZ = pythia.particleData.m0(23)
pythia.settings.parm("Beams:eCM", mZ)
# parton level first
pythia.readString("HadronLevel:all=off")
pythia.init()

In [ ]:
# print the banner first
fj.ClusterSequence.print_banner()
print()
# set up our jet definition and a jet selector
jet_R0 = 1.0
jet_def = fj.JetDefinition(fj.antikt_algorithm, jet_R0)
print(jet_def)

In [ ]:
# hadron level acceptamce
max_eta_hadron = 10.
from pyjetty.mputils import pwarning
pwarning('max eta for particles after hadronization set to', max_eta_hadron)
parts_selector_h = fj.SelectorAbsEtaMax(max_eta_hadron)
jet_selector = fj.SelectorPtMin(1.0) & fj.SelectorPtMax(100.0) & fj.SelectorAbsEtaMax(max_eta_hadron - 1.05 * jet_R0)

max_eta_parton = max_eta_hadron + 3. * jet_R0
pwarning('max eta for partons set to', max_eta_parton)
parts_selector_p = fj.SelectorAbsEtaMax(max_eta_parton)


In [ ]:
#output
outf = ROOT.TFile('lep_pythia8_out.root', 'recreate')
outf.cd()
t = ROOT.TTree('t', 't')
from pyjetty.mputils import RTreeWriter
tw = RTreeWriter(tree=t)
hpt_parton = ROOT.TH1F('pt_parton_level', 'pt_parton_level', 50, 0, 50)
hpt_hadron = ROOT.TH1F('pt_hadron_level', 'pt_hadron_level', 50, 0, 50)
hpt_hadron.SetLineColor(3)

In [ ]:
# event loop
nev = 10000
import tqdm
for iev in tqdm.tqdm(range(nev)):
    if not pythia.next():
        continue

    #select particles
    parts_pythia_p = pythiafjext.vectorize_select(pythia, [pythiafjext.kFinal], 0, True)
    parts_pythia_p_selected = parts_selector_p(parts_pythia_p)

    hstatus = pythia.forceHadronLevel()
    if not hstatus:
        pwarning('forceHadronLevel false event', iev)
        continue
    # parts_pythia_h = pythiafjext.vectorize_select(pythia, [pythiafjext.kHadron, pythiafjext.kCharged])
    parts_pythia_h = pythiafjext.vectorize_select(pythia, [pythiafjext.kFinal], 0, True)
    parts_pythia_h_selected = parts_selector_h(parts_pythia_h)

    parts_pythia_hch = pythiafjext.vectorize_select(pythia, [pythiafjext.kFinal, pythiafjext.kCharged], 0, True)
    parts_pythia_hch_selected = parts_selector_h(parts_pythia_hch)

    # jet finder is just an extra - not needed for generation
    jets_p = fj.sorted_by_pt(jet_def(parts_pythia_p))
    jets_h = fj.sorted_by_pt(jet_def(parts_pythia_h))
    jets_ch_h = fj.sorted_by_pt(jet_selector(jet_def(parts_pythia_hch)))

    _c = [hpt_parton.Fill(j.perp()) for j in jets_p]
    _c = [hpt_hadron.Fill(j.perp()) for j in jets_h]

In [ ]:
pythia.stat()

In [ ]:
# %jsroot on
canvas = ROOT.TCanvas("cpt","cpt",400,400)
hpt_parton.Draw()
hpt_hadron.Draw('same')
canvas.SetLogy()
canvas.BuildLegend()
canvas.Draw()

In [ ]:
outf.Write()
outf.Close()